## Random Forest Classifier

In [1]:
import pandas as pd
import plotly.express as plt
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt


In [2]:
df =pd.read_csv("Travel.csv")
df

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Single,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Single,3.0,0,5,0,2.0,Executive,20289.0


### Data Cleaning

In [3]:
## chekcing null values
df.isnull().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [4]:
import numpy as np
ind =0
for i in df.columns:
    if type(df.loc[:, i])==object:
        print(i)


    ind +=1

In [5]:
df.loc[:, "MonthlyIncome"]

0       20993.0
1       20130.0
2       17090.0
3       17909.0
4       18468.0
         ...   
4883    26576.0
4884    21212.0
4885    31820.0
4886    20289.0
4887    24041.0
Name: MonthlyIncome, Length: 4888, dtype: float64

In [6]:
df["Gender"].value_counts()

Gender
Male       2916
Female     1817
Fe Male     155
Name: count, dtype: int64

In [7]:
df["Gender"].replace("Fe Male", "Female", inplace =True)

C:\Users\faiqi\AppData\Local\Temp\ipykernel_10172\1375468140.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Gender"].replace("Fe Male", "Female", inplace =True)


In [8]:
df["Gender"].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [9]:
df["MaritalStatus"].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [10]:
df["MaritalStatus"].replace("Single", "Unmarried", inplace=True)

C:\Users\faiqi\AppData\Local\Temp\ipykernel_10172\3994745670.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["MaritalStatus"].replace("Single", "Unmarried", inplace=True)


In [11]:
df["MaritalStatus"].value_counts()

MaritalStatus
Married      2340
Unmarried    1598
Divorced      950
Name: count, dtype: int64

### Imputing Null Values

In [12]:
# checking cols with null values
features_with_na =[f for f in df.columns if df[f].isnull().sum()>=1]
features_with_na

['Age',
 'TypeofContact',
 'DurationOfPitch',
 'NumberOfFollowups',
 'PreferredPropertyStar',
 'NumberOfTrips',
 'NumberOfChildrenVisiting',
 'MonthlyIncome']

In [13]:
df

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Unmarried,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Unmarried,3.0,0,5,0,2.0,Executive,20289.0


In [14]:


#Age
df.Age.fillna(df.Age.median(), inplace=True)

#TypeofContract
df.TypeofContact.fillna(df.TypeofContact.mode()[0], inplace=True)

#DurationOfPitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(), inplace=True)

#NumberOfFollowups
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0], inplace=True)

#PreferredPropertyStar
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0], inplace=True)

#NumberOfTrips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(), inplace=True)

#NumberOfChildrenVisiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0], inplace=True)

#MonthlyIncome
df.MonthlyIncome.fillna(df.MonthlyIncome.median(), inplace=True)




C:\Users\faiqi\AppData\Local\Temp\ipykernel_10172\700678725.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Age.fillna(df.Age.median(), inplace=True)
C:\Users\faiqi\AppData\Local\Temp\ipykernel_10172\700678725.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [15]:
df.drop("CustomerID", axis=1, inplace=True)

## Feature Engineering

In [16]:
# create new column for feature
df['TotalVisiting'] = df['NumberOfPersonVisiting'] + df['NumberOfChildrenVisiting']
df.drop(columns=['NumberOfPersonVisiting', 'NumberOfChildrenVisiting'], axis=1, inplace=True)

### Train Test Split

In [17]:
X =df.drop("ProdTaken", axis=1)
Y =df["ProdTaken"]

In [18]:
Y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=42)

## Column Transformer

In [20]:
cat =X.select_dtypes(include ="object").columns
num =X.select_dtypes(exclude="object").columns
# no need standard scaler in this case actually
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
standscaler =StandardScaler()
onehotencoder =OneHotEncoder(drop ="first")
preprocessor =ColumnTransformer([
    ("OneHotEncoder", onehotencoder, cat),
    ("StandardScaler", standscaler, num)
])


In [21]:
# applying on training data
X_train =preprocessor.fit_transform(X_train)
X_train

array([[ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.38224537, -0.77415132],
       [ 1.        ,  0.        ,  1.        , ...,  0.78296635,
        -0.4597992 ,  0.64361526],
       [ 1.        ,  1.        ,  0.        , ...,  0.78296635,
        -0.24519557, -0.06526803],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.36057591,  0.64361526],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.25279888,  0.64361526],
       [ 0.        ,  0.        ,  1.        , ..., -1.2771941 ,
        -1.08251091, -1.48303461]])

In [22]:
# applying on test data
X_test =preprocessor.transform(X_test)
X_test

array([[ 0.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.73751038, -0.77415132],
       [ 1.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.6704111 , -0.06526803],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.4208322 , -0.77415132],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.78296635,
         0.69001249,  0.64361526],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.22827818, -0.77415132],
       [ 1.        ,  1.        ,  0.        , ...,  0.78296635,
        -0.44611323,  2.06138184]])

## Training Random Forest Classifier Model

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [24]:
models ={"Random Forest": RandomForestClassifier(), "Decison Tree":DecisionTreeClassifier() }
for i in range(len(list(models))):
    model =list(models.values())[i]


    
    model.fit(X_train, Y_train)


    # making pred
    y_pred_train =model.predict(X_train)
    y_pred_test  =model.predict(X_test)


    print(list(models.keys())[i])
    print("Model Performance for Trainning Data")
    print("Accuracy: ", accuracy_score(Y_train, y_pred_train))
    print("F1 Score: ", f1_score(Y_train, y_pred_train))
    print("Precision Score", precision_score(Y_train, y_pred_train))
    print("Recall Score", recall_score(Y_train, y_pred_train))
    print("Roc Auc Score: ", roc_auc_score(Y_train, y_pred_train))

    print("Model Performance for Testing Data")
    print("Accuracy: ", accuracy_score(Y_test, y_pred_test))
    print("F1 Score: ", f1_score(Y_test, y_pred_test))
    print("Precision Score", precision_score(Y_test, y_pred_test))
    print("Recall Score", recall_score(Y_test, y_pred_test))
    print("Roc Auc Score: ", roc_auc_score(Y_test, y_pred_test))




Random Forest
Model Performance for Trainning Data
Accuracy:  1.0
F1 Score:  1.0
Precision Score 1.0
Recall Score 1.0
Roc Auc Score:  1.0
Model Performance for Testing Data
Accuracy:  0.9284253578732107
F1 Score:  0.78125
Precision Score 0.9689922480620154
Recall Score 0.6544502617801047
Roc Auc Score:  0.8246838348290613
Decison Tree
Model Performance for Trainning Data
Accuracy:  1.0
F1 Score:  1.0
Precision Score 1.0
Recall Score 1.0
Roc Auc Score:  1.0
Model Performance for Testing Data
Accuracy:  0.9263803680981595
F1 Score:  0.8054054054054054
Precision Score 0.8324022346368715
Recall Score 0.7801047120418848
Roc Auc Score:  0.870992635563509


## HyperParameter Tunning

In [25]:
params ={"max_depth": [5, 8, 15, None, 10],
         "max_features": [5, 7, "auto", 8] ,
         "min_samples_split": [2, 8, 15, 20],
         "n_estimators": [100, 200, 500, 1000]}

In [26]:
# model list for hyperparamter tunning
modelcv =[("RF", RandomForestClassifier(), params)]

In [27]:
from sklearn.model_selection import RandomizedSearchCV
bestparams ={}
for name, model, para in modelcv:
    rd = RandomizedSearchCV(estimator=model, param_distributions=para, n_iter=100, cv =3, verbose=2, n_jobs=-1)
    rd.fit(X_train, Y_train)
    bestparams[name] = rd.best_params_

for model, bestparames in bestparams.items():
    print(model)
    print(bestparames)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\faiqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
78 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\faiqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\faiqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-pac

RF
{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}


In [28]:
y_pred =rd.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, y_pred))

0.9366053169734151


In [30]:
# training model with hyperparameters
models ={"Random Forest": RandomForestClassifier(n_estimators =100, min_samples_split = 2, max_features = 7,max_depth =15) }
for i in range(len(list(models))):
    model =list(models.values())[i]


    
    model.fit(X_train, Y_train)


    # making pred
    y_pred_train =model.predict(X_train)
    y_pred_test  =model.predict(X_test)


    print("Model Performance for Testing Data")
    print("Accuracy: ", accuracy_score(Y_test, y_pred_test))
    print("F1 Score: ", f1_score(Y_test, y_pred_test))
    print("Precision Score", precision_score(Y_test, y_pred_test))
    print("Recall Score", recall_score(Y_test, y_pred_test))
    print("Roc Auc Score: ", roc_auc_score(Y_test, y_pred_test))


Model Performance for Testing Data
Accuracy:  0.9304703476482618
F1 Score:  0.7926829268292683
Precision Score 0.948905109489051
Recall Score 0.680628272251309
Roc Auc Score:  0.8358668680189201


## Random Forest Regression

In [130]:
df =pd.read_csv("cardekho_imputated.csv")
df

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,19537,Hyundai i10,Hyundai,i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5,250000
15407,19540,Maruti Ertiga,Maruti,Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7,925000
15408,19541,Skoda Rapid,Skoda,Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5,425000
15409,19542,Mahindra XUV500,Mahindra,XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7,1225000


## Data Cleaning

In [131]:
## Droping Unnecessary Columns
df.drop(["Unnamed: 0", "car_name", "brand"], axis =1, inplace=True)


In [132]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [133]:
## Independent and Dependent features
X =df.drop("selling_price", axis =1)
Y =df["selling_price"]

In [134]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


## Train Test Split

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, train_size=0.2, random_state=42)

## Feature Encoding and Scaling

In [136]:
# too much cols so we should use LabelEncoding
len(X["model"].unique())


120

In [137]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train['model'] = le.fit_transform(X_train['model'])
X_test['model'] = le.fit_transform(X_test['model'])

In [138]:
# no need of standard scaller here
onehotencod_features =["seller_type", "fuel_type", "transmission_type"]

numericalfeatures =df.select_dtypes("number").columns
numericalfeatures =[i for i in numericalfeatures if i!="selling_price"]
from sklearn.preprocessing import OneHotEncoder,  StandardScaler
from sklearn.compose import ColumnTransformer
ohe =OneHotEncoder(drop ="first")
le =LabelEncoder()
s =StandardScaler()
trans =ColumnTransformer([("OneHotEncoder", ohe, onehotencod_features), ("StandardScaler",s, numericalfeatures )])


In [139]:
X_train =trans.fit_transform(X_train)
X_test =trans.transform(X_test)

## Training and Selecting Model

In [140]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [141]:
# evalute model
def evaluate(true, predicted):
    r2 =r2_score(true, predicted)
    mae =mean_absolute_error(true, predicted)
    mse =mean_squared_error(true, predicted)
    return f"R2 SCORE {r2}\nMEAN ABSOLUTE ERROR {mae}\nMEAN SQUARED ERROR {mse}"

In [144]:
models ={"DecisionTreeRegressor": DecisionTreeRegressor(), "RandomForestRegressor": RandomForestRegressor(), "LinearRegression": LinearRegression(),"Ridge": Ridge(), "Lasso": Lasso(), "ElasticNet": ElasticNet(), "Knn": KNeighborsRegressor() }
for i in range(len(list(models))):
    model =list(models.keys())[i]
    object =list(models.values())[i]
 
    object.fit(X_train, Y_train)
    y_pred =object.predict(X_test)
    print(model)
    print(evaluate(Y_test, y_pred))
    print(("-")*20)

DecisionTreeRegressor
R2 SCORE 0.7684337160493157
MEAN ABSOLUTE ERROR 152084.87779490094
MEAN SQUARED ERROR 186429556816.27438
--------------------
RandomForestRegressor
R2 SCORE 0.8094111786802421
MEAN ABSOLUTE ERROR 122878.83974485243
MEAN SQUARED ERROR 153439390599.47748
--------------------
LinearRegression
R2 SCORE 0.6235595942702065
MEAN ABSOLUTE ERROR 264837.0883850531
MEAN SQUARED ERROR 303064922969.9164
--------------------
Ridge
R2 SCORE 0.6235217123671521
MEAN ABSOLUTE ERROR 264731.0337329224
MEAN SQUARED ERROR 303095420960.71747
--------------------
Lasso
R2 SCORE 0.6235601733622511
MEAN ABSOLUTE ERROR 264834.4449502819
MEAN SQUARED ERROR 303064456754.0606
--------------------
ElasticNet
R2 SCORE 0.5641540666954574
MEAN ABSOLUTE ERROR 237235.43769987387
MEAN SQUARED ERROR 350891169473.7296
--------------------
Knn
R2 SCORE 0.7703909612316093
MEAN ABSOLUTE ERROR 143822.4288263444
MEAN SQUARED ERROR 184853816403.2464
--------------------


## Hyperparameter Tunning

In [152]:
params ={"max_depth": [5, 8, 15, None, 10],
         "max_features": [5,7,8, 10],
         "min_samples_split": [2, 8, 15, 20],
         "n_estimators": [100, 200, 500, 1000]
}

In [153]:
from sklearn.model_selection import RandomizedSearchCV
rd =RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=params,n_iter=100, cv =3, n_jobs=-1 , verbose =2, error_score="raise")
rd.fit(X_train, Y_train)
y_pred =rd.predict(X_test)
print(evaluate(Y_test, y_pred))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
R2 SCORE 0.8181309839921638
MEAN ABSOLUTE ERROR 120385.86188242542
MEAN SQUARED ERROR 146419243227.02173
